# 目的
OHLC図を描画して保存する。

In [ ]:
from PIL import Image
from PIL import ImageDraw
import numpy as np
import copy
import pandas as pd
import skimage
import datetime
import os

In [ ]:
def make_image(screen, bgcolor, filename, drawfunc, save=False, show=True, **kwargs):
    """
    画像ファイルを作成
    """
    img = Image.new('RGB', screen, bgcolor)

    # 間隔（5～32くらい）
#     gap = 8

    img = drawfunc(img, **kwargs)
    if show:
        plt.imshow(img, origin="normal")
        is_axis = kwargs.get("axis", True)
        if not is_axis:
            plt.axis("off")
    if save:
        img.save(filename)
    return img

def drawOhlcBox(img, ohlc, start=[0,0], width=10, barcolor=(0x00,0x00,0x00), return_next=False, **kwargs):
    """
    ohlc: (1,4) array
    start: start point (open)
    width
    barcolor
    kwargs
    """
    draw = ImageDraw.Draw(img)
    
    ### Define each start point
    ## Bar
    # start_bar[1] = start[1] + (open - open), end_bar[1] = start[1] + (close - open)
    start_bar = tuple([start[0]+width//2, start[1]])
    
    end_bar   = tuple([start_bar[0], start[1] + ohlc[3] - ohlc[0]])
    
    ## Line between high and low
    # if open > close then start_hl[1] = start_bar[1] + (high - open), end_hl[1] = end_bar[1] + (low - close)
    # if open < close then start_hl[1] = start_bar[1] + (low - open), end_hl[1] = end_bar[1] + (high - close)
    if ohlc[0] >= ohlc[3]:
        start_hl = tuple([start_bar[0], start_bar[1] + (ohlc[1] - ohlc[0])])
        end_hl   = tuple([start_bar[0], end_bar[1] + (ohlc[2] - ohlc[3])])
    else:
        start_hl = tuple([start_bar[0], start_bar[1] + (ohlc[2] - ohlc[0])])
        end_hl   = tuple([start_bar[0], end_bar[1] + (ohlc[1] - ohlc[3])])
    
    ### Draw
    if start_bar[1] == end_bar[1]:
        _end_bar = tuple([end_bar[0], end_bar[1]+1])
        draw.line((start_bar, _end_bar), barcolor, width)
    else:
        draw.line((start_bar, end_bar), barcolor, width)
    draw.line((start_hl, end_hl), barcolor, 1)
    
#     ### Joes
#     joeDistanceTop = kwargs.get("joe_distance_top", 10)
#     joeDistanceBottom = kwargs.get("joe_distance_bottom", joeDistanceTop)
#     joeLength = kwargs.get("joe_length", width)
#     joeWidth = kwargs.get("joe_width", 1)
#     joeColor = kwargs.get("joe_color", barcolor)
    
#     # Top joe
#     start = tuple([topleft[0]+(width-joeLength+1)//2, topleft[1]-joeDistanceTop])
#     end = tuple([start[0]+joeLength-1, start[1]])
#     draw.line((start, end), joeColor, joeWidth)
    
#     # Connection to bar
#     start = tuple([start_bar[0], start_bar[1]-joeDistanceTop])
#     draw.line((start, start_bar), joeColor, joeWidth)
    
#     # Bottom joe
#     start = tuple([topleft[0]+(width-joeLength+1)//2, end_bar[1]+joeDistanceBottom])
#     end = tuple([start[0]+joeLength-1, start[1]])
#     draw.line((start, end), joeColor, joeWidth)
    
#     # Connection to bar
#     end = tuple([end_bar[0], end_bar[1]+joeDistanceBottom])
#     draw.line((end, end_bar), joeColor, joeWidth)
    
    if not return_next:
        return img
    else:
        start_next = [start[0] + width, end_bar[1]]
        return img, start_next
    
def drawOhlcBoxes(img, ohlc=None, start=[0, 0], width=11, barcolors=(0x00,0x00,0x00), **kwargs):
    """
    topleft_startは上ひげのtopleftになる
    """
    if ohlc is None:
        raise ValueError("ohlc must be inputted.")
    elif not isinstance(ohlc, np.ndarray):
        raise TypeError("ohlc must be a (N, 4) array.")
    if not isinstance(barcolors, list):
        bar_colors = [barcolors] * len(ohlc)
    else:
        bar_colors = barcolors
    
    k_wargs = copy.deepcopy(kwargs)
    if k_wargs.get("width") is not None:
        del k_wargs["width"]
    if k_wargs.get("height") is not None:
        del k_wargs["height"]
    if k_wargs.get("barcolors") is not None:
        del k_wargs["barcolors"]
    
    start_next = start
    for ii in range(len(ohlc)):
        start_ohlc = [start_next[0], ohlc[ii, 0]]
        img, start_next = drawOhlcBox(img, ohlc[ii], start_ohlc, width, bar_colors[ii], True, **k_wargs)
        
    return img

def saveOhlcImages(h, w, nbr_of_ohlc, bgcolor, barcolors, stop=0):
    screen = (h, w) # Image用
    for ii in range(len(ohlc)-nbr_of_ohlc+1):
        buff = ohlc[ii:ii+nbr_of_ohlc]
        ohlc_bar = np.array((buff - buff.min()) * h / (buff.max() - buff.min()), dtype=int)
        fldrpath = "../images/{0}/".format(datetime.datetime.now().strftime("%Y%m%d"))
        if not os.path.exists(fldrpath):
            try:
                os.makedirs(fldrpath)
            except:
                pass
        filename = fldrpath + "image-ohlc-{0:05d}.png".format(ii)
        img = make_image(screen, bgcolor, filename, drawOhlcBoxes, 
                         ohlc=ohlc_bar, start=[0, ohlc_bar[0,0]], width=w//nbr_of_ohlc, 
                         barcolors=barcolors, axis=False, save=True, show=False)
        if ii == stop - 1:
            break

In [ ]:
data = pd.read_csv("../data/FXTF/USDJPY-cd1_20170806_k030.csv")
ohlc = data[["open", "high", "low", "close"]].as_matrix()
len(ohlc)

In [ ]:
h, w = 100, 100 # Imageのサイズ
nbr_of_ohlc = 10 # 描画するohlc図の数

bgcolor=(0x00, 0x00, 0x00) # 画像の背景色（RGB）
barcolors = (0xff, 0xff, 0xff) # OHLC図の色 (RGB)

saveOhlcImages(h, w, nbr_of_ohlc, bgcolor, barcolors)